<a href="https://colab.research.google.com/github/luisalfonso634/C2-G13-m/blob/main/ANALISIS_AUDIENCIA_TECHO_15_al_19MAY23_act21may23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CARGAR DRIVE Y ENTORNO**

In [61]:
!pip install pipenv
!pip install chart-studio
!pip install -q bqplot
!pip install -q pingouin
!pip install xlsxwriter



import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats
from matplotlib.patches import Rectangle
import seaborn as sns
from google.colab import files
from google.colab import drive
from datetime import datetime, timedelta
from pandas import DataFrame
import statsmodels.formula.api as sm
import chart_studio.plotly as py

drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **HORAS DE TRABAJO** 

Mayo: 4075,67
Horas de Trabajo hasta 19MAY:169

In [62]:
def obtener_datos_telemarketing(resumen_por_camp_aud_anun):
    horas_telemarketing = float(input("¿Cuántas horas de Telemarketing se trabajaron durante este período? "))
    valor_hora_telemarketing = float(input("¿Cuál es el valor de la hora de Telemarketing? "))
    # Cantidad Total de Leads
    TOTAL_LEADS = resumen_por_camp_aud_anun['IDINTERNO'].sum()

    # Realizar la multiplicación de horas_telemarketing y valor_hora_telemarketing
    total_telemarketing = horas_telemarketing * valor_hora_telemarketing

    Costo_por_lead = total_telemarketing / TOTAL_LEADS

    # Agregar la columna con el resultado al dataset resumen_por_camp_aud_anun
    resumen_por_camp_aud_anun['Costo TMK'] = resumen_por_camp_aud_anun['IDINTERNO'] * Costo_por_lead

    return resumen_por_camp_aud_anun, Costo_por_lead

resumen_por_camp_aud_anun_actualizado, costo_por_lead = obtener_datos_telemarketing(resumen_por_camp_aud_anun)


¿Cuántas horas de Telemarketing se trabajaron durante este período? 169
¿Cuál es el valor de la hora de Telemarketing? 4075


# **ANALISIS AUDIENCIA TECHO**

## **Cargar Archivos**

In [196]:
#Carga todos los leads
dfleads = pd.read_excel('/content/drive/MyDrive/TECHO 15 al 19 mayo/LEADS15_18MAY.xlsx')

#Carga Ultimo Estado
dfULTIMO_ESTADO= pd.read_excel('/content/drive/MyDrive/TECHO 15 al 19 mayo/Ultimo Estado.xlsx')

#Carga todas las Gestiones
dfGestionesTodas = pd.read_excel('/content/drive/MyDrive/TECHO 15 al 19 mayo/Todas_las_Gestiones_(Base_-_Lote).xlsx')

# Carga las Bases de los Reportes de Facebook
dfANUNCIOS = pd.read_excel('/content/drive/MyDrive/TECHO 15 al 19 mayo/FACEBOOKdel12al18MAY.xlsx')

#Carga Clave de Tipificacion
dftipificacion = pd.read_excel('/content/drive/MyDrive/TECHO AL 14 MAY/Tipificacion.xlsx')

#Carga las Donaciones
dfDonaciones = pd.read_excel('/content/drive/MyDrive/TECHO 15 al 19 mayo/Reporte_DONACIONES.xlsx')

ruta_salida_gen = '/content/drive/MyDrive/TECHO AUDIENCIAS'


## **TODAS LAS GESTIONES**

In [197]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['fechanacimiento','LOTE', 'nombre','apellido','TXTFORMULARIO','tipodocumento','TXTOBSE','telefono','sexo','sexo1', 'nrodocumento', 'domicilio', 'codigo postal',
                       'localidad', 'provincia', 'provincia1','telefono1','telefono2', 'TXTAUDIENCIA','TXTAVISO','telefono3','email2', 'TXTIDFACEBOOK',
                       'TMLOBS']

# Eliminar las columnas del DataFrame
df_todas_gestiones = dfGestionesTodas.drop(columnas_a_eliminar, axis=1)

In [198]:
df_todas_gestiones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2091 entries, 0 to 2090
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   TXTLISTA               0 non-null      float64       
 1   IDINTERNO              2091 non-null   int64         
 2   email1                 2091 non-null   object        
 3   VENDOR                 2091 non-null   int64         
 4   TXTCAMPAÑA             2081 non-null   object        
 5   TXTSEGMENTACION        2081 non-null   object        
 6   TXTNOMBREANUNCIO       2081 non-null   object        
 7   Monto                  14 non-null     float64       
 8   Frecuencia             14 non-null     object        
 9   Medio_Pago             14 non-null     object        
 10  Fecha de Modificacion  1781 non-null   datetime64[ns]
 11  ESTADO                 1781 non-null   object        
 12  CAMPAÑA                1781 non-null   float64       
 13  USU

In [199]:
dftipificacion.head()

,ESTADO,CONTACTACION,VALOR CONTACTO
0,43 - Contestador / No contesta,NO CONTACTADO,0
1,29 - Llamar mas adelante,CONTACTADO,1
2,48 - Rellamar,NO CONTACTADO,0
3,41 - Colabora de otra forma,CONTACTADO,1
4,33 - Problemas Económicos,CONTACTADO,1


In [200]:
# Realizar el merge
df_todas_gestiones = df_todas_gestiones.merge(dftipificacion, on='ESTADO', how='left')

# Verificar el resultado
#print(df_todas_gestiones.head())


In [201]:
df_todas_gestiones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2091 entries, 0 to 2090
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   TXTLISTA               0 non-null      float64       
 1   IDINTERNO              2091 non-null   int64         
 2   email1                 2091 non-null   object        
 3   VENDOR                 2091 non-null   int64         
 4   TXTCAMPAÑA             2081 non-null   object        
 5   TXTSEGMENTACION        2081 non-null   object        
 6   TXTNOMBREANUNCIO       2081 non-null   object        
 7   Monto                  14 non-null     float64       
 8   Frecuencia             14 non-null     object        
 9   Medio_Pago             14 non-null     object        
 10  Fecha de Modificacion  1781 non-null   datetime64[ns]
 11  ESTADO                 1781 non-null   object        
 12  CAMPAÑA                1781 non-null   float64       
 13  USU

In [202]:
# Contar las repeticiones de cada ID INTERNO
conteo_id_interno = df_todas_gestiones['IDINTERNO'].value_counts()


# Imprimir los IDINTERNO en líneas separadas
# Obtener el número de IDINTERNO únicos
num_id_interno = df_todas_gestiones['IDINTERNO'].nunique()

# Imprimir el número de IDINTERNO únicos
print("Número de IDINTERNO diferentes:", num_id_interno)



# Calcular la sumatoria de VALOR CONTACTO por cada IDINTERNO
suma_valor_contacto = df_todas_gestiones.groupby('IDINTERNO')['VALOR CONTACTO'].sum()

Número de IDINTERNO diferentes: 1058


In [203]:
# Contar las repeticiones de cada IDINTERNO
conteo_id_interno = df_todas_gestiones['IDINTERNO'].value_counts()

# Calcular la sumatoria de VALOR CONTACTO por cada IDINTERNO
suma_valor_contacto = df_todas_gestiones.groupby('IDINTERNO')['VALOR CONTACTO'].sum()

In [204]:
import pandas as pd

# Contar las repeticiones de cada IDINTERNO
conteo_id_interno = df_todas_gestiones['IDINTERNO'].value_counts()

# Calcular la sumatoria de VALOR CONTACTO por cada IDINTERNO
suma_valor_contacto = df_todas_gestiones.groupby('IDINTERNO')['VALOR CONTACTO'].sum()

# Crear la tabla con los resultados
tabla_resultados = pd.DataFrame({'IDINTERNO': conteo_id_interno.index, 'Tipificaciones Realizadas': conteo_id_interno.values, 'Contactos Realizados': [suma_valor_contacto.get(id_interno, 0) for id_interno in conteo_id_interno.index]})

# Agregar la nueva columna "Estado de Contacto"
tabla_resultados['Estado de Contacto'] = tabla_resultados['Contactos Realizados'].apply(lambda x: 'CONTACTADO' if x > 0 else 'NO CONTACTADO')
tabla_resultados['Contactacion Historica'] = tabla_resultados['Estado de Contacto'].apply(lambda x: 1 if x == 'CONTACTADO' else 0)

# Agregar las cuatro columnas adicionales
df_adicional = df_todas_gestiones[['IDINTERNO', 'TXTCAMPAÑA', 'TXTSEGMENTACION', 'TXTNOMBREANUNCIO']]

# Reemplazar caracteres en las columnas TXTCAMPAÑA y TXTSEGMENTACION
df_adicional['TXTCAMPAÑA'] = df_adicional['TXTCAMPAÑA'].str.replace('±', 'ñ')
df_adicional['TXTCAMPAÑA'] = df_adicional['TXTCAMPAÑA'].str.replace('Ý', 'í')
df_adicional['TXTCAMPAÑA'] = df_adicional['TXTCAMPAÑA'].str.replace('¾', 'ó')
df_adicional['TXTCAMPAÑA'] = df_adicional['TXTCAMPAÑA'].str.replace('PolÝtica', 'Política')
df_adicional['TXTCAMPAÑA'] = df_adicional['TXTCAMPAÑA'].str.replace('EconomÝa', 'Economía')
df_adicional['TXTCAMPAÑA'] = df_adicional['TXTCAMPAÑA'].str.replace('Construcci¾n', 'Construcción')
df_adicional['TXTSEGMENTACION'] = df_adicional['TXTSEGMENTACION'].str.replace('±', 'ñ')
df_adicional['TXTSEGMENTACION'] = df_adicional['TXTSEGMENTACION'].str.replace('¾', 'ó')
df_adicional['TXTSEGMENTACION'] = df_adicional['TXTSEGMENTACION'].str.replace('Ý', 'í')
df_adicional['TXTSEGMENTACION'] = df_adicional['TXTSEGMENTACION'].str.replace('PolÝtica', 'Política')
df_adicional['TXTSEGMENTACION'] = df_adicional['TXTSEGMENTACION'].str.replace('EconomÝa', 'Economía')
df_adicional['TXTSEGMENTACION'] = df_adicional['TXTSEGMENTACION'].str.replace('Construcci¾n', 'Construcción')

tabla_resultados = pd.merge(tabla_resultados, df_adicional.drop_duplicates(subset='IDINTERNO'), on='IDINTERNO', how='left')



# Especificar la ruta de salida del archivo Excel
ruta_salida = '/content/drive/MyDrive/TECHO 15 al 19 mayo/Gestiones_Todas_Contactacion.xlsx'

# Exportar la tabla como un archivo de Excel
tabla_resultados.to_excel(ruta_salida, index=False)

print("El archivo Gestiones_Todas_Contactacion_2.xlsx se ha guardado exitosamente.")


<ipython-input-204-94dd08f6e4c4>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adicional['TXTCAMPAÑA'] = df_adicional['TXTCAMPAÑA'].str.replace('±', 'ñ')
<ipython-input-204-94dd08f6e4c4>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adicional['TXTCAMPAÑA'] = df_adicional['TXTCAMPAÑA'].str.replace('Ý', 'í')
<ipython-input-204-94dd08f6e4c4>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

El archivo Gestiones_Todas_Contactacion_2.xlsx se ha guardado exitosamente.


In [205]:
tabla_resultados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 1057
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   IDINTERNO                  1058 non-null   int64  
 1   Tipificaciones Realizadas  1058 non-null   int64  
 2   Contactos Realizados       1058 non-null   float64
 3   Estado de Contacto         1058 non-null   object 
 4   Contactacion Historica     1058 non-null   int64  
 5   TXTCAMPAÑA                 1050 non-null   object 
 6   TXTSEGMENTACION            1050 non-null   object 
 7   TXTNOMBREANUNCIO           1050 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 74.4+ KB


In [206]:
# Contactados por Audiencia Todas las Gestiones
conteo_contactados_todas_gestiones = df_todas_gestiones[df_todas_gestiones['CONTACTACION'] == 'CONTACTADO'].groupby('TXTSEGMENTACION').count()['CONTACTACION']

# Imprimir el conteo de contactados por cada valor único en "TXTSEGMENTACION"
print("Conteo de contactados por cada valor único en 'TXTSEGMENTACION':")
print(conteo_contactados_todas_gestiones)

Conteo de contactados por cada valor único en 'TXTSEGMENTACION':
TXTSEGMENTACION
Audiencia Alto Poder Adquisitivo          18
Audiencia Arquitectura y Construcci¾n     50
Audiencia Bancos                           9
Audiencia Compras online                   7
Audiencia LookAlike Donantes              30
Audiencia PolÝtica y EconomÝa            109
Audiencia Profesionales                    3
Audiencia Real Estate                     73
Name: CONTACTACION, dtype: int64


## **ULTIMO ESTADO**

In [207]:
# Verifica el resultado
df_ultimo_estado=dfULTIMO_ESTADO

In [208]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['fechanacimiento','LOTE', 'nombre','apellido','tipodocumento','TXTOBSE','telefono','sexo','sexo1', 'nrodocumento', 'domicilio', 'codigo postal',
                       'localidad', 'provincia', 'provincia1','telefono1','telefono2','telefono3','email2', 'TXTIDFACEBOOK',
                       'TMLOBS']

# Eliminar las columnas del DataFrame
df_ultimo_estado = df_ultimo_estado.drop(columnas_a_eliminar, axis=1)


In [209]:
# Realizar el merge
df_ultimo_estado = df_ultimo_estado.merge(dftipificacion, on='ESTADO', how='left')



In [210]:
df_ultimo_estado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 1057
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   IDINTERNO              1058 non-null   int64         
 1   email1                 1058 non-null   object        
 2   Fecha de Modificacion  1058 non-null   datetime64[ns]
 3   ESTADO                 1058 non-null   object        
 4   VENDOR                 1058 non-null   int64         
 5   TXTCAMPAÑA             1050 non-null   object        
 6   TXTAUDIENCIA           1050 non-null   float64       
 7   TXTAVISO               1050 non-null   float64       
 8   TXTFORMULARIO          1058 non-null   object        
 9   TXTSEGMENTACION        1050 non-null   object        
 10  TXTNOMBREANUNCIO       1050 non-null   object        
 11  CONTACTACION           771 non-null    object        
 12  VALOR CONTACTO         771 non-null    float64       
dtypes: 

In [211]:
import pandas as pd

# Contar las repeticiones de cada IDINTERNO
conteo_id_interno_ue = df_ultimo_estado['IDINTERNO'].value_counts()

# Calcular la sumatoria de VALOR CONTACTO por cada IDINTERNO
suma_valor_contacto_ue = df_ultimo_estado.groupby('IDINTERNO')['VALOR CONTACTO'].sum()

# Crear la tabla con los resultados
tabla_resultados_ue = pd.DataFrame({'IDINTERNO': conteo_id_interno_ue.index, 'Tipificaciones Realizadas': conteo_id_interno_ue.values, 'Contactos Realizados': [suma_valor_contacto_ue.get(id_interno, 0) for id_interno in conteo_id_interno_ue.index]})

# Agregar la nueva columna "Estado de Contacto"
tabla_resultados_ue['Estado de Contacto'] = tabla_resultados_ue['Contactos Realizados'].apply(lambda x: 'CONTACTADO' if x > 0 else 'NO CONTACTADO')
tabla_resultados_ue['Contactacion Historica'] = tabla_resultados_ue['Estado de Contacto'].apply(lambda x: 1 if x == 'CONTACTADO' else 0)

# Agregar las cuatro columnas adicionales
df_adicional2 = df_ultimo_estado[['IDINTERNO', 'TXTCAMPAÑA', 'TXTSEGMENTACION', 'TXTNOMBREANUNCIO']]

# Reemplazar caracteres en las columnas TXTCAMPAÑA y TXTSEGMENTACION
df_adicional2['TXTCAMPAÑA'] = df_adicional2['TXTCAMPAÑA'].str.replace('±', 'ñ')
df_adicional2['TXTCAMPAÑA'] = df_adicional2['TXTCAMPAÑA'].str.replace('Ý', 'í')
df_adicional2['TXTCAMPAÑA'] = df_adicional2['TXTCAMPAÑA'].str.replace('¾', 'ó')
df_adicional2['TXTCAMPAÑA'] = df_adicional2['TXTCAMPAÑA'].str.replace('PolÝtica', 'Política')
df_adicional2['TXTCAMPAÑA'] = df_adicional2['TXTCAMPAÑA'].str.replace('EconomÝa', 'Economía')
df_adicional2['TXTCAMPAÑA'] = df_adicional2['TXTCAMPAÑA'].str.replace('Construcci¾n', 'Construcción')
df_adicional2['TXTSEGMENTACION'] = df_adicional2['TXTSEGMENTACION'].str.replace('±', 'ñ')
df_adicional2['TXTSEGMENTACION'] = df_adicional2['TXTSEGMENTACION'].str.replace('¾', 'ó')
df_adicional2['TXTSEGMENTACION'] = df_adicional2['TXTSEGMENTACION'].str.replace('Ý', 'í')
df_adicional2['TXTSEGMENTACION'] = df_adicional2['TXTSEGMENTACION'].str.replace('PolÝtica', 'Política')
df_adicional2['TXTSEGMENTACION'] = df_adicional2['TXTSEGMENTACION'].str.replace('EconomÝa', 'Economía')
df_adicional2['TXTSEGMENTACION'] = df_adicional2['TXTSEGMENTACION'].str.replace('Construcci¾n', 'Construcción')

tabla_resultados_ue = pd.merge(tabla_resultados_ue, df_adicional2.drop_duplicates(subset='IDINTERNO'), on='IDINTERNO', how='left')




<ipython-input-211-9a8a7336d2d4>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adicional2['TXTCAMPAÑA'] = df_adicional2['TXTCAMPAÑA'].str.replace('±', 'ñ')
<ipython-input-211-9a8a7336d2d4>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adicional2['TXTCAMPAÑA'] = df_adicional2['TXTCAMPAÑA'].str.replace('Ý', 'í')
<ipython-input-211-9a8a7336d2d4>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



In [212]:
tabla_resultados_ue.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 1057
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   IDINTERNO                  1058 non-null   int64  
 1   Tipificaciones Realizadas  1058 non-null   int64  
 2   Contactos Realizados       1058 non-null   float64
 3   Estado de Contacto         1058 non-null   object 
 4   Contactacion Historica     1058 non-null   int64  
 5   TXTCAMPAÑA                 1050 non-null   object 
 6   TXTSEGMENTACION            1050 non-null   object 
 7   TXTNOMBREANUNCIO           1050 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 74.4+ KB


In [213]:
resumen_por_camp_aud_anun_ue = tabla_resultados_ue.groupby(['TXTCAMPAÑA', 'TXTSEGMENTACION', 'TXTNOMBREANUNCIO']).agg({'Contactacion Historica': 'sum'})

# Especificar la ruta de salida del archivo Excel
ruta_salida = ruta_salida_gen + '/2Resumen_por_Campaña_Audiencia_Anuncio.xlsx'

# Exportar el resumen como un archivo de Excel
resumen_por_camp_aud_anun_ue.to_excel(ruta_salida)
resumen_por_camp_aud_anun_ue

Contactacion Historica
TXTCAMPAÑA                          TXTSEGMENTACION                       TXTNOMBREANUNCIO                        
Campaña Arquitectura y Construcción Audiencia Arquitectura y Construcción Anuncio Techo 1                       43
                                                                          Anuncio Techo 2                        0
Campaña Bancos                      Audiencia Bancos                      Anuncio Techo 1                        1
                                                                          Anuncio Techo 2                        8
Campaña Compras online              Audiencia Compras online              Anuncio Techo 1                        4
                                                                          Anuncio Techo 2                        3
Campaña LookAlike Donantes          Audiencia LookAlike Donantes          Anuncio Techo 1                       30
                                                                          Anuncio Techo 2                        1
Campaña Nivel Alto                  Audiencia Alto Poder Adquisitivo      Anuncio Techo 1                        8
                                                                          Anuncio Techo 2                        6
Campaña Política y Economía         Audiencia Política y Economía         Anuncio Techo 1                       82
                                                                          Anuncio Techo 2                        6
Campaña Profesionales               Audiencia Profesionales               Anuncio Techo 1                        2
                                                                          Anuncio Techo 2                        1
Campaña Real Estate                 Audiencia Real Estate                 Anuncio Techo 1                       52
                                                                          Anuncio Techo 2                       16

## **DONACIONES**

In [214]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['Nombre Campaña','estado','Tipo Campaña','FB_CAMPAÑA','FB_ANUNCIO','FB_AUDIENCIA','TXTIDFACEBOOK','nombre','apellido', 'telefono','TXTAUDIENCIA','fecha modif','sexo','sexo1','fechanacimiento','TXTAVISO','nrodocumento','tipodocumento','domicilio', 'codigo postal', 'localidad','provincia','telefono1','telefono2','telefono3','email2','TMLOBS','TXTFORMULARIO','TXTOBSE','titular tarjeta','tipo de pago','nro tarjeta','tipo tarjeta','duracion','vto tarjeta','operador','list_name','email TL']

# Eliminar las columnas del DataFrame
dfDonaciones = dfDonaciones.drop(columnas_a_eliminar, axis=1)

In [215]:
dfDonaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   IDINTERNO         5 non-null      int64         
 1   email1            5 non-null      object        
 2   TXTCAMPAÑA        5 non-null      object        
 3   TXTSEGMENTACION   5 non-null      object        
 4   TXTNOMBREANUNCIO  5 non-null      object        
 5   fecha llamado     5 non-null      datetime64[ns]
 6   monto             5 non-null      int64         
dtypes: datetime64[ns](1), int64(2), object(4)
memory usage: 408.0+ bytes


In [216]:
dfDonaciones.head()

,IDINTERNO,email1,TXTCAMPAÑA,TXTSEGMENTACION,TXTNOMBREANUNCIO,fecha llamado,monto
0,37493,melthommdq@hotmail.com,Campa±a LookAlike Donantes,Audiencia LookAlike Donantes,Anuncio Techo 1,2023-05-16 10:43:04.920,1000
1,37719,bb1651542@gmail.com,Campa±a Real Estate,Audiencia Real Estate,Anuncio Techo 1,2023-05-17 10:06:19.887,1000
2,37856,sntejedor@estudiosnt.com.ar,Campa±a LookAlike Donantes,Audiencia LookAlike Donantes,Anuncio Techo 1,2023-05-18 11:04:11.560,800
3,52590,adrianasalinasvicetn@gmail.com,Campa±a LookAlike Donantes,Audiencia LookAlike Donantes,Anuncio Techo 1,2023-05-18 20:31:12.837,1500
4,52715,laurafabi.sanchez1986@gmail.com,Campa±a Compras online,Audiencia Compras online,Anuncio Techo 2,2023-05-19 11:46:06.530,500


In [217]:
tabla_resultados_ue_donaciones = tabla_resultados_ue.merge(dfDonaciones[['IDINTERNO', 'email1', 'TXTCAMPAÑA', 'TXTSEGMENTACION', 'TXTNOMBREANUNCIO', 'monto', 'fecha llamado']], on='IDINTERNO', how='left')


In [218]:
#UÑTIMO ESTADO + MONTO DONACIONES
tabla_resultados_ue_donaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 1057
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   IDINTERNO                  1058 non-null   int64         
 1   Tipificaciones Realizadas  1058 non-null   int64         
 2   Contactos Realizados       1058 non-null   float64       
 3   Estado de Contacto         1058 non-null   object        
 4   Contactacion Historica     1058 non-null   int64         
 5   TXTCAMPAÑA_x               1050 non-null   object        
 6   TXTSEGMENTACION_x          1050 non-null   object        
 7   TXTNOMBREANUNCIO_x         1050 non-null   object        
 8   email1                     5 non-null      object        
 9   TXTCAMPAÑA_y               5 non-null      object        
 10  TXTSEGMENTACION_y          5 non-null      object        
 11  TXTNOMBREANUNCIO_y         5 non-null      object        
 12  monto 

In [219]:
tabla_resultados_ue_donaciones = tabla_resultados_ue_donaciones.filter(['IDINTERNO', 'monto'])


In [220]:
tabla_resultados_ue_donaciones.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 1057
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   IDINTERNO  1058 non-null   int64  
 1   monto      5 non-null      float64
dtypes: float64(1), int64(1)
memory usage: 24.8 KB


In [221]:
#Cantidad de Donaciones
cantidad_valores_mayor_cero = (tabla_resultados_ue_donaciones['monto'] > 0).sum()
cantidad_valores_mayor_cero

5

In [222]:
#Monto Total de las Donaciones
suma_monto = tabla_resultados_ue_donaciones['monto'].sum()
suma_monto

4800.0

In [223]:
#HAceindo el Merge de los ultimos estados con las donaciones, lo que quiero son los IDINTERNO y los Montos de las donaciones

# Realizando el merge
df_merged = tabla_resultados_ue.merge(dfDonaciones, on='IDINTERNO')

# Renombrando la columna
df_merged = df_merged.rename(columns={'monto': 'monto donacion'})


In [224]:
# Renombrando la columna en dfDonaciones antes del merge
dfDonaciones = dfDonaciones.rename(columns={'monto': 'monto donacion'})

# Realizando el merge con una unión 'left'
df_merged = tabla_resultados_ue.merge(dfDonaciones, on='IDINTERNO', how='left')


In [225]:
#Tengo los ultimos estados con el monte de las donaciones y los IDINTERNO
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 1057
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   IDINTERNO                  1058 non-null   int64         
 1   Tipificaciones Realizadas  1058 non-null   int64         
 2   Contactos Realizados       1058 non-null   float64       
 3   Estado de Contacto         1058 non-null   object        
 4   Contactacion Historica     1058 non-null   int64         
 5   TXTCAMPAÑA_x               1050 non-null   object        
 6   TXTSEGMENTACION_x          1050 non-null   object        
 7   TXTNOMBREANUNCIO_x         1050 non-null   object        
 8   email1                     5 non-null      object        
 9   TXTCAMPAÑA_y               5 non-null      object        
 10  TXTSEGMENTACION_y          5 non-null      object        
 11  TXTNOMBREANUNCIO_y         5 non-null      object        
 12  fecha 

In [226]:
df_resultado = tabla_resultados.merge(df_merged[['IDINTERNO', 'monto donacion']], on='IDINTERNO', how='left')
df_resultado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 1057
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   IDINTERNO                  1058 non-null   int64  
 1   Tipificaciones Realizadas  1058 non-null   int64  
 2   Contactos Realizados       1058 non-null   float64
 3   Estado de Contacto         1058 non-null   object 
 4   Contactacion Historica     1058 non-null   int64  
 5   TXTCAMPAÑA                 1050 non-null   object 
 6   TXTSEGMENTACION            1050 non-null   object 
 7   TXTNOMBREANUNCIO           1050 non-null   object 
 8   monto donacion             5 non-null      float64
dtypes: float64(2), int64(3), object(4)
memory usage: 82.7+ KB


## **REPORTES DE FACEBOOK**

In [227]:
# Carga las Bases de los Reportes de Facebook

df_anuncios=dfANUNCIOS

In [228]:
# Lista de columnas a eliminar
columnas_a_eliminar = ['Entrega del anuncio','Puja', 'Tipo de puja','Alcance','Presupuesto del conjunto de anuncios','Tipo de presupuesto del conjunto de anuncios','Último cambio significativo','Configuración de atribución','Indicador de resultado','Impresiones','Clasificación de calidad','Clasificación del porcentaje de interacción', 'Clasificación del porcentaje de conversiones', 'Finalización']

# Eliminar las columnas del DataFrame
df_anuncios = df_anuncios.drop(columnas_a_eliminar, axis=1)

In [229]:
df_anuncios

,Inicio del informe,Fin del informe,Nombre del anuncio,Nombre del conjunto de anuncios,Resultados,Costo por resultados,Importe gastado (ARS)
0,2023-05-12,2023-05-18,Anuncio Techo 1,Audiencia LookAlike Donantes,139.0,60.414173,8397.57
1,2023-05-12,2023-05-18,Anuncio Techo 2,Audiencia LookAlike Donantes,NaN,NaN,25.92
2,2023-05-12,2023-05-18,Anuncio Techo 1,Audiencia Política y Economía,244.0,22.187869,5413.84
3,2023-05-12,2023-05-18,Anuncio Techo 2,Audiencia Política y Economía,2.0,9.770000,19.54
4,2023-05-12,2023-05-18,Anuncio Techo 1,Audiencia Real Estate,153.0,33.056536,5057.65
5,2023-05-12,2023-05-18,Anuncio Techo 2,Audiencia Real Estate,13.0,27.557692,358.25
6,2023-05-12,2023-05-18,Anuncio Techo 1,Audiencia Arquitectura y Construcción,132.0,40.404697,5333.42
7,2023-05-12,2023-05-18,Anuncio Techo 2,Audiencia Arquitectura y Construcción,2.0,47.870000,95.74
8,2023-05-12,2023-05-18,Anuncio Techo 2,Audiencia Alto Poder Adquisitivo,3.0,167.033333,501.10
9,2023-05-12,2023-05-18,Anuncio Techo 1,Audiencia Alto Poder Adquisitivo,45.0,111.918889,5036.35


In [230]:
# Cambiar los nombres de las columnas
df_anuncios.rename(columns={'Nombre del conjunto de anuncios': 'TXTSEGMENTACION', 'Nombre del anuncio': 'TXTNOMBREANUNCIO'}, inplace=True)

In [231]:
df_anuncios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Inicio del informe     16 non-null     datetime64[ns]
 1   Fin del informe        16 non-null     datetime64[ns]
 2   TXTNOMBREANUNCIO       16 non-null     object        
 3   TXTSEGMENTACION        16 non-null     object        
 4   Resultados             15 non-null     float64       
 5   Costo por resultados   15 non-null     float64       
 6   Importe gastado (ARS)  16 non-null     float64       
dtypes: datetime64[ns](2), float64(3), object(2)
memory usage: 1.0+ KB


In [232]:

tabla_resultados = df_resultado.merge(df_anuncios[['TXTNOMBREANUNCIO', 'TXTSEGMENTACION', 'Costo por resultados']], on=['TXTNOMBREANUNCIO', 'TXTSEGMENTACION'], how='left')
tabla_resultados

,IDINTERNO,Tipificaciones Realizadas,Contactos Realizados,Estado de Contacto,Contactacion Historica,TXTCAMPAÑA,TXTSEGMENTACION,TXTNOMBREANUNCIO,monto donacion,Costo por resultados
0,37413,19,1.0,CONTACTADO,1,Campaña Real Estate,Audiencia Real Estate,Anuncio Techo 2,NaN,27.557692
1,37431,12,0.0,NO CONTACTADO,0,Campaña Política y Economía,Audiencia Política y Economía,Anuncio Techo 1,NaN,22.187869
2,37546,12,3.0,CONTACTADO,1,Campaña Nivel Alto,Audiencia Alto Poder Adquisitivo,Anuncio Techo 2,NaN,167.033333
3,37434,12,0.0,NO CONTACTADO,0,Campaña Arquitectura y Construcción,Audiencia Arquitectura y Construcción,Anuncio Techo 1,NaN,40.404697
4,37528,12,0.0,NO CONTACTADO,0,Campaña Política y Economía,Audiencia Política y Economía,Anuncio Techo 1,NaN,22.187869
...,...,...,...,...,...,...,...,...,...,...
1053,37988,1,0.0,NO CONTACTADO,0,Campaña Política y Economía,Audiencia Política y Economía,Anuncio Techo 1,NaN,22.187869
1054,37989,1,0.0,NO CONTACTADO,0,Campaña Política y Economía,Audiencia Política y Economía,Anuncio Techo 1,NaN,22.187869
1055,37990,1,0.0,NO CONTACTADO,0,Campaña Política y Economía,Audiencia Política y Economía,Anuncio Techo 1,NaN,22.187869
1056,37991,1,0.0,NO CONTACTADO,0,Campaña Real Estate,Audiencia Real Estate,Anuncio Techo 1,NaN,33.056536


In [233]:
# Realiza una operación de combinación (merge) entre dos DataFrames: tabla_resultados y df_resultado. LE QUITE MONTO DONACION OJOJOJOJOJ
tabla_resultados = tabla_resultados.merge(df_resultado[['IDINTERNO']], on='IDINTERNO', how='left')

# Especificar la ruta de salida del archivo Excel
#

# Exportar la tabla como un archivo de Excel
#tabla_resultados.to_excel(ruta_salida, index=False)


In [234]:
tabla_resultados.head(1)

,IDINTERNO,Tipificaciones Realizadas,Contactos Realizados,Estado de Contacto,Contactacion Historica,TXTCAMPAÑA,TXTSEGMENTACION,TXTNOMBREANUNCIO,monto donacion,Costo por resultados
0,37413,19,1.0,CONTACTADO,1,Campaña Real Estate,Audiencia Real Estate,Anuncio Techo 2,NaN,27.557692


In [235]:
# Agregar la nueva columna "Cantidad de Donaciones"
tabla_resultados['Cantidad de Donaciones'] = tabla_resultados['monto donacion'].apply(lambda x: 1 if x > 1 else 0)

# Especificar la ruta de salida del archivo Excel
ruta_salida = ruta_salida_gen + '/Gestiones_Todas_Contactacion_INV_FBL.xlsx'
# Exportar la tabla como un archivo de Excel
tabla_resultados.to_excel(ruta_salida, index=False)


In [236]:
tabla_resultados

,IDINTERNO,Tipificaciones Realizadas,Contactos Realizados,Estado de Contacto,Contactacion Historica,TXTCAMPAÑA,TXTSEGMENTACION,TXTNOMBREANUNCIO,monto donacion,Costo por resultados,Cantidad de Donaciones
0,37413,19,1.0,CONTACTADO,1,Campaña Real Estate,Audiencia Real Estate,Anuncio Techo 2,NaN,27.557692,0
1,37431,12,0.0,NO CONTACTADO,0,Campaña Política y Economía,Audiencia Política y Economía,Anuncio Techo 1,NaN,22.187869,0
2,37546,12,3.0,CONTACTADO,1,Campaña Nivel Alto,Audiencia Alto Poder Adquisitivo,Anuncio Techo 2,NaN,167.033333,0
3,37434,12,0.0,NO CONTACTADO,0,Campaña Arquitectura y Construcción,Audiencia Arquitectura y Construcción,Anuncio Techo 1,NaN,40.404697,0
4,37528,12,0.0,NO CONTACTADO,0,Campaña Política y Economía,Audiencia Política y Economía,Anuncio Techo 1,NaN,22.187869,0
...,...,...,...,...,...,...,...,...,...,...,...
1053,37988,1,0.0,NO CONTACTADO,0,Campaña Política y Economía,Audiencia Política y Economía,Anuncio Techo 1,NaN,22.187869,0
1054,37989,1,0.0,NO CONTACTADO,0,Campaña Política y Economía,Audiencia Política y Economía,Anuncio Techo 1,NaN,22.187869,0
1055,37990,1,0.0,NO CONTACTADO,0,Campaña Política y Economía,Audiencia Política y Economía,Anuncio Techo 1,NaN,22.187869,0
1056,37991,1,0.0,NO CONTACTADO,0,Campaña Real Estate,Audiencia Real Estate,Anuncio Techo 1,NaN,33.056536,0


In [237]:
tabla_resultados.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1058 entries, 0 to 1057
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   IDINTERNO                  1058 non-null   int64  
 1   Tipificaciones Realizadas  1058 non-null   int64  
 2   Contactos Realizados       1058 non-null   float64
 3   Estado de Contacto         1058 non-null   object 
 4   Contactacion Historica     1058 non-null   int64  
 5   TXTCAMPAÑA                 1050 non-null   object 
 6   TXTSEGMENTACION            1050 non-null   object 
 7   TXTNOMBREANUNCIO           1050 non-null   object 
 8   monto donacion             5 non-null      float64
 9   Costo por resultados       1047 non-null   float64
 10  Cantidad de Donaciones     1058 non-null   int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 99.2+ KB


In [238]:
#resumen_por_camp_aud_anun = tabla_resultados.groupby(['TXTCAMPAÑA', 'TXTSEGMENTACION', 'TXTNOMBREANUNCIO']).agg({'IDINTERNO': 'count', 'Cantidad de Donaciones': 'sum', 'Contactacion Historica': 'sum', 'monto donacion': 'sum'})
resumen_por_camp_aud_anun = tabla_resultados.groupby(['TXTCAMPAÑA', 'TXTSEGMENTACION', 'TXTNOMBREANUNCIO']).agg({'IDINTERNO': 'count', 'monto donacion': 'sum', 'Cantidad de Donaciones': 'sum', 'Contactacion Historica': 'sum','Costo por resultados':'unique'})
resumen_por_camp_aud_anun = resumen_por_camp_aud_anun.reindex(columns=['IDINTERNO', 'monto donacion', 'Cantidad de Donaciones', 'Contactacion Historica'])

# Especificar la ruta de salida del archivo Excel
ruta_salida = ruta_salida_gen + '/Resumen_por_Campaña_Audiencia_Anuncio.xlsx'
# Exportar el resumen como un archivo de Excel
resumen_por_camp_aud_anun.to_excel(ruta_salida)


In [239]:
# Filtrar los valores mayores a cero en la columna "Cantidad de Donaciones"
valores_distintos_cero = tabla_resultados.loc[tabla_resultados['Cantidad de Donaciones'] > 0, 'Cantidad de Donaciones'].value_counts()

# Imprimir el resultado
print(valores_distintos_cero)


1    5
Name: Cantidad de Donaciones, dtype: int64


In [240]:
resumen_por_camp_aud_anun = resumen_por_camp_aud_anun.merge(df_anuncios[['TXTNOMBREANUNCIO', 'TXTSEGMENTACION', 'Costo por resultados', 'Resultados']], on=['TXTNOMBREANUNCIO', 'TXTSEGMENTACION'], how='left')


In [241]:
resumen_por_camp_aud_anun

,TXTNOMBREANUNCIO,TXTSEGMENTACION,IDINTERNO,monto donacion,Cantidad de Donaciones,Contactacion Historica,Costo por resultados,Resultados
0,Anuncio Techo 1,Audiencia Arquitectura y Construcción,176,0.0,0,44,40.404697,132.0
1,Anuncio Techo 2,Audiencia Arquitectura y Construcción,3,0.0,0,0,47.870000,2.0
2,Anuncio Techo 1,Audiencia Bancos,3,0.0,0,1,113.633333,3.0
3,Anuncio Techo 2,Audiencia Bancos,18,0.0,0,8,62.612778,18.0
4,Anuncio Techo 1,Audiencia Compras online,9,0.0,0,4,90.106667,9.0
5,Anuncio Techo 2,Audiencia Compras online,8,500.0,1,3,81.687500,8.0
6,Anuncio Techo 1,Audiencia LookAlike Donantes,177,3300.0,3,29,60.414173,139.0
7,Anuncio Techo 2,Audiencia LookAlike Donantes,3,0.0,0,1,NaN,NaN
8,Anuncio Techo 1,Audiencia Alto Poder Adquisitivo,53,0.0,0,8,111.918889,45.0
9,Anuncio Techo 2,Audiencia Alto Poder Adquisitivo,14,0.0,0,6,167.033333,3.0


In [242]:
suma_total_act = resumen_por_camp_aud_anun.sum()
suma_total_act

TXTNOMBREANUNCIO          Anuncio Techo 1Anuncio Techo 2Anuncio Techo 1A...
TXTSEGMENTACION           Audiencia Arquitectura y ConstrucciónAudiencia...
IDINTERNO                                                              1050
monto donacion                                                       4800.0
Cantidad de Donaciones                                                    5
Contactacion Historica                                                  266
Costo por resultados                                            1076.713467
Resultados                                                            785.0
dtype: object

In [243]:
resumen_por_camp_aud_anun.columns


Index(['TXTNOMBREANUNCIO', 'TXTSEGMENTACION', 'IDINTERNO', 'monto donacion',
       'Cantidad de Donaciones', 'Contactacion Historica',
       'Costo por resultados', 'Resultados'],
      dtype='object')

In [244]:
resumen_por_camp_aud_anun = resumen_por_camp_aud_anun.groupby(['TXTSEGMENTACION', 'TXTNOMBREANUNCIO']).agg({'IDINTERNO': 'count', 'monto donacion': 'sum', 'Cantidad de Donaciones': 'sum', 'Contactacion Historica': 'sum', 'Costo por resultados': 'sum', 'Resultados': 'sum'})
resumen_por_camp_aud_anun.rename(columns={'IDINTERNO': 'LEADS'}, inplace=True)


suma_total = resumen_por_camp_aud_anun.sum()
suma_total, resumen_por_camp_aud_anun.shape

(LEADS                       16.000000
 monto donacion            4800.000000
 Cantidad de Donaciones       5.000000
 Contactacion Historica     266.000000
 Costo por resultados      1076.713467
 Resultados                 785.000000
 dtype: float64,
 (16, 6))

In [245]:
resumen_por_camp_aud_anun.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 16 entries, ('Audiencia Alto Poder Adquisitivo', 'Anuncio Techo 1') to ('Audiencia Real Estate', 'Anuncio Techo 2')
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   LEADS                   16 non-null     int64  
 1   monto donacion          16 non-null     float64
 2   Cantidad de Donaciones  16 non-null     int64  
 3   Contactacion Historica  16 non-null     int64  
 4   Costo por resultados    16 non-null     float64
 5   Resultados              16 non-null     float64
dtypes: float64(3), int64(3)
memory usage: 1009.0+ bytes


In [246]:
resumen_por_camp_aud_anun = tabla_resultados.groupby(['TXTCAMPAÑA', 'TXTSEGMENTACION', 'TXTNOMBREANUNCIO']).agg({'IDINTERNO': 'count', 'monto donacion': 'sum', 'Cantidad de Donaciones': 'sum', 'Contactacion Historica': 'sum'})

# Calcular los totales de cada columna
total_leads = resumen_por_camp_aud_anun['IDINTERNO'].sum()
total_monto_donacion = resumen_por_camp_aud_anun['monto donacion'].sum()
total_cantidad_donaciones = resumen_por_camp_aud_anun['Cantidad de Donaciones'].sum()
total_contactacion_historica = resumen_por_camp_aud_anun['Contactacion Historica'].sum()

# Agregar las nuevas columnas con los porcentajes
resumen_por_camp_aud_anun['% LEADS'] = resumen_por_camp_aud_anun['IDINTERNO'].apply(lambda x: (x / total_leads) * 100)
resumen_por_camp_aud_anun['% monto donacion'] = resumen_por_camp_aud_anun['monto donacion'].apply(lambda x: (x / total_monto_donacion) * 100)
resumen_por_camp_aud_anun['% Cantidad de Donaciones'] = resumen_por_camp_aud_anun['Cantidad de Donaciones'].apply(lambda x: (x / total_cantidad_donaciones) * 100)
resumen_por_camp_aud_anun['% Contactacion Historica'] = resumen_por_camp_aud_anun['Contactacion Historica'].apply(lambda x: (x / total_contactacion_historica) * 100)

# Especificar la ruta de salida del archivo Excel
ruta_salida = ruta_salida_gen + '/Resumen_por_Campaña_Audiencia_Anuncio2.xlsx'


# Crear el objeto ExcelWriter
writer = pd.ExcelWriter(ruta_salida, engine='xlsxwriter')

# Exportar el DataFrame a Excel en la hoja "Analisis de Audiencias TECHO"
resumen_por_camp_aud_anun.to_excel(writer, sheet_name='Analisis de Audiencias TECHO', index=False)

# Guardar el archivo Excel
writer.save()

print("El archivo Resumen_por_Campaña_Audiencia_Anuncio.xlsx se ha guardado exitosamente.")



# Exportar el resumen como un archivo de Excel
resumen_por_camp_aud_anun.to_excel(ruta_salida)

El archivo Resumen_por_Campaña_Audiencia_Anuncio.xlsx se ha guardado exitosamente.


<ipython-input-246-95ebc7dee0b6>:26: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


In [247]:
resumen_por_camp_aud_anun

IDINTERNO  \
TXTCAMPAÑA                          TXTSEGMENTACION                       TXTNOMBREANUNCIO              
Campaña Arquitectura y Construcción Audiencia Arquitectura y Construcción Anuncio Techo 1         176   
                                                                          Anuncio Techo 2           3   
Campaña Bancos                      Audiencia Bancos                      Anuncio Techo 1           3   
                                                                          Anuncio Techo 2          18   
Campaña Compras online              Audiencia Compras online              Anuncio Techo 1           9   
                                                                          Anuncio Techo 2           8   
Campaña LookAlike Donantes          Audiencia LookAlike Donantes          Anuncio Techo 1         177   
                                                                          Anuncio Techo 2           3   
Campaña Nivel Alto                  Audiencia Alto Poder Adquisitivo      Anuncio Techo 1          53   
                                                                          Anuncio Techo 2          14   
Campaña Política y Economía         Audiencia Política y Economía         Anuncio Techo 1         317   
                                                                          Anuncio Techo 2           8   
Campaña Profesionales               Audiencia Profesionales               Anuncio Techo 1           7   
                                                                          Anuncio Techo 2           7   
Campaña Real Estate                 Audiencia Real Estate                 Anuncio Techo 1         214   
                                                                          Anuncio Techo 2          33   

                                                                                            monto donacion  \
TXTCAMPAÑA                          TXTSEGMENTACION                       TXTNOMBREANUNCIO                   
Campaña Arquitectura y Construcción Audiencia Arquitectura y Construcción Anuncio Techo 1              0.0   
                                                                          Anuncio Techo 2              0.0   
Campaña Bancos                      Audiencia Bancos                      Anuncio Techo 1              0.0   
                                                                          Anuncio Techo 2              0.0   
Campaña Compras online              Audiencia Compras online              Anuncio Techo 1              0.0   
                                                                          Anuncio Techo 2            500.0   
Campaña LookAlike Donantes          Audiencia LookAlike Donantes          Anuncio Techo 1           3300.0   
                                                                          Anuncio Techo 2              0.0   
Campaña Nivel Alto                  Audiencia Alto Poder Adquisitivo      Anuncio Techo 1              0.0   
                                                                          Anuncio Techo 2              0.0   
Campaña Política y Economía         Audiencia Política y Economía         Anuncio Techo 1              0.0   
                                                                          Anuncio Techo 2              0.0   
Campaña Profesionales               Audiencia Profesionales               Anuncio Techo 1              0.0   
                                                                          Anuncio Techo 2              0.0   
Campaña Real Estate                 Audiencia Real Estate                 Anuncio Techo 1           1000.0   
                                                                          Anuncio Techo 2              0.0   

                                                                                            Cantidad de Donaciones  \
TXTCAMPAÑA                          TXTSEGMENTACION                       TXTNOMBREANUNCIO            

In [248]:
TOTAL_LEADS = resumen_por_camp_aud_anun['IDINTERNO'].sum()
TOTAL_LEADS

1050

In [249]:
resumen_por_camp_aud_anun = resumen_por_camp_aud_anun.reset_index()


In [250]:
resumen_por_camp_aud_anun.columns

Index(['TXTCAMPAÑA', 'TXTSEGMENTACION', 'TXTNOMBREANUNCIO', 'IDINTERNO',
       'monto donacion', 'Cantidad de Donaciones', 'Contactacion Historica',
       '% LEADS', '% monto donacion', '% Cantidad de Donaciones',
       '% Contactacion Historica'],
      dtype='object')

In [251]:
costo_por_lead

655.8809523809524

In [252]:
resumen_por_camp_aud_anun_actualizado

,TXTCAMPAÑA,TXTSEGMENTACION,TXTNOMBREANUNCIO,IDINTERNO,monto donacion,Cantidad de Donaciones,Contactacion Historica,% LEADS,% monto donacion,% Cantidad de Donaciones,% Contactacion Historica,Costo TMK
0,Campaña Arquitectura y Construcción,Audiencia Arquitectura y Construcción,Anuncio Techo 1,176,0.0,0,44,16.761905,0.000000,0.0,16.541353,115435.047619
1,Campaña Arquitectura y Construcción,Audiencia Arquitectura y Construcción,Anuncio Techo 2,3,0.0,0,0,0.285714,0.000000,0.0,0.000000,1967.642857
2,Campaña Bancos,Audiencia Bancos,Anuncio Techo 1,3,0.0,0,1,0.285714,0.000000,0.0,0.375940,1967.642857
3,Campaña Bancos,Audiencia Bancos,Anuncio Techo 2,18,0.0,0,8,1.714286,0.000000,0.0,3.007519,11805.857143
4,Campaña Compras online,Audiencia Compras online,Anuncio Techo 1,9,0.0,0,4,0.857143,0.000000,0.0,1.503759,5902.928571
5,Campaña Compras online,Audiencia Compras online,Anuncio Techo 2,8,500.0,1,3,0.761905,10.416667,20.0,1.127820,5247.047619
6,Campaña LookAlike Donantes,Audiencia LookAlike Donantes,Anuncio Techo 1,177,3300.0,3,29,16.857143,68.750000,60.0,10.902256,116090.928571
7,Campaña LookAlike Donantes,Audiencia LookAlike Donantes,Anuncio Techo 2,3,0.0,0,1,0.285714,0.000000,0.0,0.375940,1967.642857
8,Campaña Nivel Alto,Audiencia Alto Poder Adquisitivo,Anuncio Techo 1,53,0.0,0,8,5.047619,0.000000,0.0,3.007519,34761.690476
9,Campaña Nivel Alto,Audiencia Alto Poder Adquisitivo,Anuncio Techo 2,14,0.0,0,6,1.333333,0.000000,0.0,2.255639,9182.333333


In [253]:
resumen_por_camp_aud_anun_actualizado.info()
INFORME_FINAL=resumen_por_camp_aud_anun_actualizado

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   TXTCAMPAÑA                16 non-null     object 
 1   TXTSEGMENTACION           16 non-null     object 
 2   TXTNOMBREANUNCIO          16 non-null     object 
 3   IDINTERNO                 16 non-null     int64  
 4   monto donacion            16 non-null     float64
 5   Cantidad de Donaciones    16 non-null     int64  
 6   Contactacion Historica    16 non-null     int64  
 7   % LEADS                   16 non-null     float64
 8   % monto donacion          16 non-null     float64
 9   % Cantidad de Donaciones  16 non-null     float64
 10  % Contactacion Historica  16 non-null     float64
 11  Costo TMK                 16 non-null     float64
dtypes: float64(6), int64(3), object(3)
memory usage: 1.6+ KB


In [254]:
# Especificar la ruta de salida del archivo Excel
ruta_salida = ruta_salida_gen + '/Resumen_por_Campaña_Audiencia_Anuncio3.xlsx'

# Crear el objeto ExcelWriter
writer = pd.ExcelWriter(ruta_salida, engine='xlsxwriter')

# Exportar el DataFrame a Excel en la hoja "Analisis de Audiencias TECHO"
INFORME_FINAL.to_excel(writer, sheet_name='Analisis de Audiencias TECHO', index=False)

# Guardar el archivo Excel
writer.save()

print("El archivo resumen_por_camp_aud_anun_actualizado se ha guardado exitosamente.")

<ipython-input-254-45e905f851f9>:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


El archivo resumen_por_camp_aud_anun_actualizado se ha guardado exitosamente.


In [255]:
suma_total_act = INFORME_FINAL.sum()
suma_total_act

TXTCAMPAÑA                  Campaña Arquitectura y ConstrucciónCampaña Arq...
TXTSEGMENTACION             Audiencia Arquitectura y ConstrucciónAudiencia...
TXTNOMBREANUNCIO            Anuncio Techo 1Anuncio Techo 2Anuncio Techo 1A...
IDINTERNO                                                                1050
monto donacion                                                         4800.0
Cantidad de Donaciones                                                      5
Contactacion Historica                                                    266
% LEADS                                                                 100.0
% monto donacion                                                        100.0
% Cantidad de Donaciones                                                100.0
% Contactacion Historica                                                100.0
Costo TMK                                                            688675.0
dtype: object

In [256]:
INFORME_FINAL.columns

Index(['TXTCAMPAÑA', 'TXTSEGMENTACION', 'TXTNOMBREANUNCIO', 'IDINTERNO',
       'monto donacion', 'Cantidad de Donaciones', 'Contactacion Historica',
       '% LEADS', '% monto donacion', '% Cantidad de Donaciones',
       '% Contactacion Historica', 'Costo TMK'],
      dtype='object')

In [257]:
df_anuncios.columns

Index(['Inicio del informe', 'Fin del informe', 'TXTNOMBREANUNCIO',
       'TXTSEGMENTACION', 'Resultados', 'Costo por resultados',
       'Importe gastado (ARS)'],
      dtype='object')

In [261]:
# Realizar el merge entre INFORME_FINAL y df_anuncios
INFORME_FINAL2 = INFORME_FINAL.merge(df_anuncios[['TXTNOMBREANUNCIO', 'TXTSEGMENTACION', 'Resultados', 'Costo por resultados', 'Importe gastado (ARS)']], 
                                on=['TXTNOMBREANUNCIO', 'TXTSEGMENTACION'], how='left')

# Seleccionar las columnas deseadas
INFORME_FINAL2 =INFORME_FINAL2[['TXTCAMPAÑA', 'TXTSEGMENTACION', 'TXTNOMBREANUNCIO', 'IDINTERNO','% LEADS', 'monto donacion', '% monto donacion',
                                        'Cantidad de Donaciones','% Cantidad de Donaciones', 'Contactacion Historica', '% Contactacion Historica','Resultados', 
                                        'Costo por resultados', 'Importe gastado (ARS)','Costo TMK']]
INFORME_FINAL2

,TXTCAMPAÑA,TXTSEGMENTACION,TXTNOMBREANUNCIO,IDINTERNO,% LEADS,monto donacion,% monto donacion,Cantidad de Donaciones,% Cantidad de Donaciones,Contactacion Historica,% Contactacion Historica,Resultados,Costo por resultados,Importe gastado (ARS),Costo TMK
0,Campaña Arquitectura y Construcción,Audiencia Arquitectura y Construcción,Anuncio Techo 1,176,16.761905,0.0,0.000000,0,0.0,44,16.541353,132.0,40.404697,5333.42,115435.047619
1,Campaña Arquitectura y Construcción,Audiencia Arquitectura y Construcción,Anuncio Techo 2,3,0.285714,0.0,0.000000,0,0.0,0,0.000000,2.0,47.870000,95.74,1967.642857
2,Campaña Bancos,Audiencia Bancos,Anuncio Techo 1,3,0.285714,0.0,0.000000,0,0.0,1,0.375940,3.0,113.633333,340.90,1967.642857
3,Campaña Bancos,Audiencia Bancos,Anuncio Techo 2,18,1.714286,0.0,0.000000,0,0.0,8,3.007519,18.0,62.612778,1127.03,11805.857143
4,Campaña Compras online,Audiencia Compras online,Anuncio Techo 1,9,0.857143,0.0,0.000000,0,0.0,4,1.503759,9.0,90.106667,810.96,5902.928571
5,Campaña Compras online,Audiencia Compras online,Anuncio Techo 2,8,0.761905,500.0,10.416667,1,20.0,3,1.127820,8.0,81.687500,653.50,5247.047619
6,Campaña LookAlike Donantes,Audiencia LookAlike Donantes,Anuncio Techo 1,177,16.857143,3300.0,68.750000,3,60.0,29,10.902256,139.0,60.414173,8397.57,116090.928571
7,Campaña LookAlike Donantes,Audiencia LookAlike Donantes,Anuncio Techo 2,3,0.285714,0.0,0.000000,0,0.0,1,0.375940,NaN,NaN,25.92,1967.642857
8,Campaña Nivel Alto,Audiencia Alto Poder Adquisitivo,Anuncio Techo 1,53,5.047619,0.0,0.000000,0,0.0,8,3.007519,45.0,111.918889,5036.35,34761.690476
9,Campaña Nivel Alto,Audiencia Alto Poder Adquisitivo,Anuncio Techo 2,14,1.333333,0.0,0.000000,0,0.0,6,2.255639,3.0,167.033333,501.10,9182.333333


In [265]:


ruta_salida = ruta_salida_gen + '/Resumen_por_Campaña_Audiencia_Anuncio5.xlsx'


# Crear el objeto ExcelWriter
writer = pd.ExcelWriter(ruta_salida, engine='xlsxwriter')

# Exportar el DataFrame a Excel en la hoja "Analisis de Audiencias TECHO"
INFORME_FINAL2.to_excel(writer, sheet_name='Analisis de Audiencias TECHO', index=False)

# Guardar el archivo Excel
writer.save()

print("El archivo Resumen_por_Campaña_Audiencia_Anuncio.xlsx se ha guardado exitosamente.")



# Exportar el resumen como un archivo de Excel
INFORME_FINAL2.to_excel(ruta_salida)

El archivo Resumen_por_Campaña_Audiencia_Anuncio.xlsx se ha guardado exitosamente.


<ipython-input-265-18272e737764>:11: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
